# CNN

## 1、引子（边界检测）
![](https://p3-juejin.byteimg.com/tos-cn-i-k3u1fbpfcp/21ce411c78234ffc8d1b9d59d5d389df~tplv-k3u1fbpfcp-watermark.awebp)

假设我们有这样一张图片，大小是8x8，像素越大，表示颜色越亮。图片中间的分界线是我们要检测的边界，怎么检测这个边界呢？我们可以设计一个
* 滤波器（filter，也称为kernel），大小3x3。

然后我们用这个filter，往图上盖，计算过程就是下图
![](https://p3-juejin.byteimg.com/tos-cn-i-k3u1fbpfcp/5c8cdc8c7b2e4b26889c7895075496ef~tplv-k3u1fbpfcp-watermark.awebp)
覆盖一块跟filter一样大的区域之后，对应元素相乘，然后求和。计算一个区域之后，就向其他区域挪动，接着计算，直到把原图片的每一个角落都覆盖到了为止。这个过程就是**卷积**。

## 诶？！发现了什么？
这个图片，中间颜色浅，两边颜色深，这说明咱们的原图片中间的边界，在这里被反映出来了!从上面这个例子中，我们发现，我们可以通过设计特定的filter，让它去跟图片做卷积，就可以识别出图片中的某些特征，比如边界。

上面的例子是检测竖直边界，我们也可以设计出检测水平边界的，只用把刚刚的filter旋转90°即可。对于其他的特征，理论上只要我们经过精细的设计，总是可以设计出合适的filter的。我们的CNN（convolutional neural network），主要就是通过一个个的filter，不断地提取特征，从局部的特征到总体的特征，从而进行图像识别等等功能。
## 如何设计filter呢？
首先，我们都不一定清楚对于一大推图片，我们需要识别哪些特征，其次，就算知道了有哪些特征，想真的去设计出对应的filter，恐怕也并非易事，要知道，特征的数量可能是成千上万的。其实学过神经网络之后，我们就知道，这些filter，根本就不用我们去设计，每个filter中的各个数字，不就是参数吗，我们可以通过大量的数据，来 让机器自己去“学习”这些参数嘛。这，就是CNN的原理。


## 2、CNN的基本概念
###  2.1 padding（填白）
从上面的引子中，我们可以知道，原图像在经过filter卷积之后，变小了，从(8,8)变成了(6,6)。假设我们再卷一次，那大小就变成了(4,4)了。

这样有啥问题呢？ 主要有两个问题： - 每次卷积，图像都缩小，这样卷不了几次就没了；相比于图片中间的点，图片边缘的点在卷积中被计算的次数很少。这样的话，边缘的信息就易于丢失。

为了解决这个问题，我们可以采用padding的方法。我们每次卷积前，先给图片周围都补一圈空白，让卷积之后图片跟原来一样大，同时，原来的边缘也被计算了更多次。
![](https://pic3.zhimg.com/80/v2-f03987b7625b534bd268d0eeb2303586_720w.jpg)

（比如，我们把(8,8)的图片给补成(10,10)，那么经过(3,3)的filter之后，就是(8,8)，没有变。）

我们把上面这种“让卷积之后的大小不变”的padding方式，称为 “Same”方式， 把不经过任何填白的，称为 “Valid”方式。这个是我们在使用一些框架的时候，需要设置的超参数。
* SAME：让卷积之后的大小不变。
* Valid：让卷积之后变小。

### 2.2 stride（步长）
前面我们所介绍的卷积，都是默认步长是1，但实际上，我们可以设置步长为其他的值。 比如，对于(8,8)的输入，我们用(3,3)的filter， 如果stride=1，则输出为(6,6); 如果stride=2，则输出为(3,3);（这里例子举得不大好，除不断就向下取整）
### 2.3 pooling 池化
这个pooling，是为了提取一定区域的主要特征，并减少参数数量，防止模型过拟合。 比如下面的MaxPooling，采用了一个2×2的窗口，并取stride=2：
![](https://pic1.zhimg.com/80/v2-26cd15a3bc3002d5882696c0dc9c4e38_720w.jpg)
除了MaxPooling,还有AveragePooling，顾名思义就是取那个区域的平均值。
* MaxPooling
* AveragePooling
### 2.4 对多通道（channels）图片的卷积
这个需要单独提一下。彩色图像，一般都是RGB三个通道（channel）的，因此输入数据的维度一般有三个：**（长，宽，通道）**。 比如一个28×28的RGB图片，维度就是(28,28,3)。
前面的引子中，输入图片是2维的(8,8)，filter是(3,3)，输出也是2维的(6,6)。

如果输入图片是三维的呢（即增多了一个channels），比如是(8,8,3)，这个时候，我们的filter的维度就要变成(3,3,3)了，它的 最后一维要跟输入的channel维度一致。 这个时候的卷积，**是三个channel的所有元素对应相乘后求和**，也就是之前是9个乘积的和，现在是27个乘积的和。因此，输出的维度并不会变化。还是(6,6)。

但是，一般情况下，我们会 使用多了filters同时卷积，比如，如果我们同时使用4个filter的话，那么 输出的维度则会变为(6,6,4)。

![](https://pic4.zhimg.com/80/v2-ed142569eafbf2d3060b3248fd03a733_720w.jpg)
（上图说明：![]()）

其实，如果套用我们学过的神经网络的符号来看待CNN的话，
* 我们的输入图片就是X，shape=(8,8,3);
* 4个filters其实就是第一层神金网络的参数W1,，shape=(3,3,3,4),这个4是指有4个filters;
* 我们的输出，就是Z1，shape=(6,6,4);
* 后面其实还应该有一个激活函数，比如relu，经过激活后，Z1变为A1，shape=(6,6,4);

![](https://pic4.zhimg.com/80/v2-aa9c4c1cc67cd853e56093ffb2116707_720w.jpg)


## 3、CNN的结构组成
上面我们已经知道了卷积（convolution）、池化（pooling）以及填白（padding）是怎么进行的，接下来我们就来看看CNN的整体结构，它包含了3种层（layer）：
### 3.1 Convolutional layer（卷积层--CONV）
由滤波器filters和激活函数构成。 一般要设置的超参数包括filters的数量、大小、步长，以及padding是“valid”还是“same”。当然，还包括选择什么激活函数。
### 3.2 Pooling layer （池化层--POOL）
这里里面没有参数需要我们学习，因为这里里面的参数都是我们设置好了，要么是Maxpooling，要么是Averagepooling。 需要指定的超参数，包括是Max还是average，窗口大小以及步长。 通常，我们使用的比较多的是Maxpooling,而且一般取大小为(2,2)步长为2的filter，这样，经过pooling之后，输入的长宽都会缩小2倍，channels不变。
### 3.3 Fully Connected layer（全连接层--FC）
这个前面没有讲，是因为这个就是我们最熟悉的家伙，就是我们之前学的神经网络中的那种最普通的层，就是一排神经元。因为这一层是每一个单元都和前一层的每一个单元相连接，所以称之为“全连接”。 这里要指定的超参数，无非就是神经元的数量，以及激活函数。
![](https://pic2.zhimg.com/80/v2-399e0b195f7a2bda06c61d5e8b5dfb75_720w.jpg)

这里需要说明的是，在经过数次卷积和池化之后，我们 最后会先将多维的数据进行“扁平化”，也就是把 (height,width,channel)的数据压缩成长度为 height × width × channel 的一维数组，然后再与 FC层连接，这之后就跟普通的神经网络无异了。

可以从图中看到，随着网络的深入，我们的图像（严格来说中间的那些不能叫图像了，但是为了方便，还是这样说吧）越来越小，但是channels却越来越大了。在图中的表示就是长方体面对我们的面积越来越小，但是长度却越来越长了。

## 4、卷积神经网络 VS. 传统神经网络
其实现在回过头来看，CNN跟我们之前学习的神经网络，也没有很大的差别。 传统的神经网络，其实就是多个FC层叠加起来。 CNN，无非就是把FC改成了CONV和POOL，就是把传统的由一个个神经元组成的layer，变成了由filters组成的layer。

那么，为什么要这样变？有什么好处？ 具体说来有两点：

### 